In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math


df=pd.read_csv(r'C:\Users\Abdulquddus\Downloads/energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [21]:
df.drop(columns=['lights','date'], inplace=True)


In [22]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scalerfit = scaler.fit_transform(df)

In [25]:
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
features_df = normalised_df.drop(columns=['Appliances'])
target = normalised_df['Appliances']

In [27]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test, = train_test_split(features_df, target, test_size=0.3, random_state=42)

In [28]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)

In [33]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
    
    #this function returns the weight of every feature
   weights = pd.Series(model.coef_, feat.columns).sort_values()

   weights_df = pd.DataFrame(weights).reset_index()
    
   weights_df.columns = ['Features', col_name]

   weights_df[col_name].round(3)
    
   return weights_df


In [34]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
lassopredicted_values = lasso_reg.predict(x_test)

In [35]:
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)
ridgepredicted_values = ridge_reg.predict(x_test)

In [38]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'lasso_weight')

In [41]:
final_weights =pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [42]:
final_weights

,Features,Linear_Model_Weight,Ridge_weight,lasso_weight
0,RH_2,-0.456698,-0.401134,-0.000000
1,T_out,-0.321860,-0.250765,0.000000
2,T2,-0.236178,-0.193880,0.000000
3,T9,-0.189941,-0.188584,-0.000000
4,RH_8,-0.157595,-0.156596,-0.000110
5,RH_out,-0.077671,-0.050541,-0.049557
6,RH_7,-0.044614,-0.046291,-0.000000
7,RH_9,-0.039800,-0.041701,-0.000000
8,T5,-0.015657,-0.020727,-0.000000
9,T1,-0.003281,-0.021549,0.000000


# STAGE B QUIZ ANSWERS

Question 1

In linear regression, L2 regularization is equivalent to imposing a:

ANSWER: GAUSSIAN PRIOR

Question 2

A best fit line relating X and Y has a R-Squared value of 0.75. How do I interpret this information?

ANSWER: 75% of the variance in Y is explained by X

Question 3

The Lasso can be interpreted as least-squares linear regression where:

ANSWER: Weights are regularized with the L1 norm

Question 4

Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

ANSWER: RH_2 and RH_1

Question 5

In the different terms of the bias-variance tradeoff, which of the following is substantially more harmful to the test error than the training error?

ANSWER: Variance

Question 6

How many coefficients do you need to estimate a simple linear regression model (One independent variable)?

ANSWER: 2(two)

Question 7

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

ANSWER: No, there will no change to the root mean squared error(RMSE)

Question 8

What can you use to find the best fit line for data in Linear Regression?

ANSWER: Least Square Error is use to find the best fit line for data in Linear Regression

Question 9

Adding more basis functions in a linear model:

ANSWER: It decreases model bias

Question 10

Cross validation:

ANSWER:It is guaranteed to prevent overfitting

Question 11

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

ANSWER: 0.64

In [45]:
#ANSWER
x = df[['T2']]
y = df[['T6']]

In [49]:
#split dataset into testing and training dataset.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [50]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

LinearRegression()

In [55]:
y_pre = linear_model.predict(x_test)

In [56]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test,y_pre)
round(r2_score, 2)

0.64

Question 12

Ridge regression:

ANSWER: Reduces variance at the expense of higher bias

Question 13

Which of the following sentence is true about outliers in Linear Regression:

ANSWER: Linear regression is sensitive to outliers

Question 14

What is the Residual Sum of Squares (in two decimal places)?

ANSWERS:

T6 = 78033.97, T2 = 10321055.64

In [64]:
#Question 14 answers
#obtain predictions
predicted_values = linear_model.predict(x_test)
predicted_values

array([[ 2.15578912],
       [10.01116055],
       [ 1.87391554],
       ...,
       [ 4.24758774],
       [ 8.69822311],
       [ 4.9893603 ]])

In [60]:
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

T6    78033.97
dtype: float64

In [62]:
#obtain predictions
predicted_values = linear_model.predict(y_test)
predicted_values

array([[-34.69865079],
       [-31.73579924],
       [-37.95927104],
       ...,
       [-16.0028032 ],
       [ -8.00119661],
       [ -9.54938191]])

In [63]:
import numpy as np
rss = np.sum(np.square(x_test - predicted_values))
round(rss, 2)

T2    10321055.64
dtype: float64

Question 15

What is the Mean Absolute Error (in two decimal places)?

ANSWER: 2.82, 12.44

In [68]:
#question 15
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

2.82

In [69]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(x_test, predicted_values)
round(mae, 2)

12.44

Question 16

What is the Root Mean Squared Error (in three decimal places)?

ANSWER: 3.63, 12.725

In [70]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

3.63

In [71]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(x_test, predicted_values))
round(rmse, 3)

12.725

Question 17

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

ANSWER:

In [72]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)